In [43]:
import torch
import pytubefix

In [ ]:
name = "Hart_aber_fair_Vierkampf_timestamped"
m4a_file = name +  '.m4a'

In [ ]:
link = "https://www.youtube.com/watch?v=YZ2G0KYkQGA"
yt = pytubefix.YouTube(link)

#For abr choose the highest available one
yt.streams.filter(only_audio=True, abr="128kbps").first().download(filename=m4a_file)

'/Users/cecilia/Desktop/Master/FSS25/Teamprojekt/debate_transcription/Hart_aber_fair_Vierkampf_timestamped.m4a'

In [45]:
from pydub import AudioSegment

wav_filename = 'debate_' + name + '.wav'

sound = AudioSegment.from_file(m4a_file, format='m4a')
file_handle = sound.export(wav_filename, format='wav')

python(69580) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(69582) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [46]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_device = torch.device(device)
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

In [47]:
with open("access_token","r") as f:
    hf_access_token = f.read()

In [48]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained(
  "pyannote/speaker-diarization-3.1",
  use_auth_token=hf_access_token)

pipeline.to(torch_device)

# run the pipeline on an audio file
diarization = pipeline(wav_filename, num_speakers=5)

speaker_file = "timestamps_" + name + ".rttm"

# dump the diarization output to disk using RTTM format
with open(speaker_file, "w") as rttm:
    diarization.write_rttm(rttm)

KeyboardInterrupt: 

In [ ]:

for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"start={turn.start:.1f}s stop={turn.end:.1f}s {speaker}")

start=0.0s stop=9.5s SPEAKER_04
start=10.2s stop=48.8s SPEAKER_01
start=36.8s stop=36.9s SPEAKER_00
start=36.9s stop=36.9s SPEAKER_03
start=36.9s stop=37.0s SPEAKER_00
start=37.0s stop=37.0s SPEAKER_03
start=37.0s stop=37.1s SPEAKER_00
start=49.0s stop=105.8s SPEAKER_01
start=65.5s stop=67.0s SPEAKER_04
start=67.0s stop=68.5s SPEAKER_00
start=105.4s stop=106.2s SPEAKER_04
start=106.2s stop=136.2s SPEAKER_01
start=135.0s stop=135.3s SPEAKER_04
start=136.0s stop=139.6s SPEAKER_04
start=139.7s stop=143.9s SPEAKER_04
start=143.9s stop=143.9s SPEAKER_03
start=143.9s stop=144.0s SPEAKER_04
start=144.0s stop=144.1s SPEAKER_03
start=144.2s stop=160.3s SPEAKER_03
start=161.0s stop=162.0s SPEAKER_03
start=162.5s stop=208.1s SPEAKER_03
start=178.7s stop=180.8s SPEAKER_00
start=208.1s stop=209.0s SPEAKER_04
start=211.0s stop=211.4s SPEAKER_04
start=212.5s stop=228.6s SPEAKER_04
start=228.6s stop=299.1s SPEAKER_00
start=250.5s stop=251.1s SPEAKER_01
start=295.0s stop=295.0s SPEAKER_02
start=295.0s 

In [ ]:
# Saving the speakers and timestamps into a file
from datetime import timedelta

# Time formatting: seconds -> hh:mm:ss
def format_time(seconds):
    return str(timedelta(seconds=round(seconds)))

# Filename for the output
speaker_file = "formatted_timestamps_" + name + ".txt"

# Minimum duration for a speach in seconds
min_duration = 1.0

# Maximum permitted break between two segments of the same speaker
max_pause = 1.0

with open(speaker_file, "w", encoding="utf-8") as file:
    merged_start = None
    merged_end = None
    merged_speaker = None

    # Iterating over the segments
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        # Calculation of the duration
        exact_duration = turn.end - turn.start
        duration = round(exact_duration, 2)

        # Only consider segments that are larger than the threshold
        if duration >= min_duration:
            # If the current speaker is the same as the previous one and the break is short enough
            if speaker == merged_speaker and (turn.start - merged_end) <= max_pause:
                # Update end time
                merged_end = turn.end
            else:
                # If there was previously a merged segment, write this segment
                if merged_speaker:
                    merged_exact_duration = merged_end - merged_start
                    start_hhmmss = format_time(merged_start)
                    end_hhmmss = format_time(merged_end)
                    file.write(f"Start: {start_hhmmss}, Dauer: {round(merged_exact_duration, 2)}s, Ende: {end_hhmmss}, Sprecher: {merged_speaker}\n")

                # Start new segment
                merged_start = turn.start
                merged_end = turn.end
                merged_speaker = speaker

    # Write last merged segment, if present
    if merged_speaker:
        merged_exact_duration = merged_end - merged_start
        start_hhmmss = format_time(merged_start)
        end_hhmmss = format_time(merged_end)
        file.write(f"Start: {start_hhmmss}, Dauer: {round(merged_exact_duration, 2)}s, Ende: {end_hhmmss}, Sprecher: {merged_speaker}\n")

In [ ]:
# Assignment of speakers to their names
speaker_names = {
    'SPEAKER_00': 'Sahra Wagenknecht (BSW)',
    'SPEAKER_01': 'Dorothee Bär (CSU)',
    'SPEAKER_02': 'Christian Lindner (FDP)',
    'SPEAKER_03': 'Jan van Aken (Die Linke)',
    'SPEAKER_04': 'Moderator Louis Klamroth'
}

with open(speaker_file, 'r') as f:
    lines = f.readlines()

# Add names to the file
new_lines = []
for line in lines:
    for speaker, speaker_name in speaker_names.items():
        if speaker in line:
            line = line.strip() + f', {speaker_name}\n'
            break 
    new_lines.append(line)

# Save changes directly in the original file
with open(speaker_file, 'w') as f:
    f.writelines(new_lines)

In [ ]:
import whisper_timestamped

# Load Whisper Timestapemd model
model = whisper_timestamped.load_model("large-v2")

# Transcription with timestamps
result = whisper_timestamped.transcribe(model, wav_filename, language="de")

# Print results
for segment in result['segments']:
    print(f"[{segment['start']:.2f} - {segment['end']:.2f}] {segment['text']}")

100%|██████████| 67170/67170 [59:08<00:00, 18.93frames/s]  

[0.00 - 4.50]  Die hart arbeitende Mitte, über die wird ganz viel gesprochen in Deutschland.
[4.92 - 7.22]  Wer ist diese hart arbeitende Mitte eigentlich?
[7.22 - 9.34]  Und was wollen Sie für die tun?
[10.14 - 12.52]  Ich glaub, das Entscheidende ist bei den Menschen,
[12.54 - 15.49]  die wirklich auf die Arbeit gehen, die jeden Tag auch wollen,
[15.49 - 17.52]  dass ihre Kinder eine gute Ausbildung bekommen.
[17.98 - 19.86]  Da spielt Bildung die alles entscheidende Rolle.
[20.66 - 22.12]  Dass sie je nach ihren Fähigkeiten
[22.66 - 24.56]  in die unterschiedlichen Schularten gehen können,
[26.30 - 28.14]  dass sie einen guten Abschluss bekommen.
[28.14 - 29.80]  Dass sie die Möglichkeit haben, so wie das früher der Fall war,
[31.14 - 32.94]  auch wenn das eine romantische Vorstellung ist,
[33.86 - 36.52]  aber dass man die Möglichkeit hat, sich Eigentum zu schaffen.
[37.30 - 37.98]  Mich beschäftigt sehr,
[38.74 - 41.38]  dass wir in Europa die niedrigste Eigentumsrate haben.
[41.8

In [ ]:
for segment in result["segments"]:
    for word in segment["words"]:
        print(f"{word['text']} - Start: {word['start']}, Ende: {word['end']}")

Die - Start: 0.0, Ende: 0.42
hart - Start: 0.42, Ende: 0.88
arbeitende - Start: 0.88, Ende: 1.42
Mitte, - Start: 1.42, Ende: 1.66
über - Start: 1.82, Ende: 1.94
die - Start: 1.94, Ende: 2.08
wird - Start: 2.08, Ende: 2.28
ganz - Start: 2.28, Ende: 2.52
viel - Start: 2.52, Ende: 2.7
gesprochen - Start: 2.7, Ende: 3.16
in - Start: 3.16, Ende: 4.04
Deutschland. - Start: 4.04, Ende: 4.5
Wer - Start: 4.92, Ende: 5.24
ist - Start: 5.24, Ende: 5.46
diese - Start: 5.46, Ende: 5.72
hart - Start: 5.72, Ende: 5.98
arbeitende - Start: 5.98, Ende: 6.44
Mitte - Start: 6.44, Ende: 6.72
eigentlich? - Start: 6.72, Ende: 7.22
Und - Start: 7.22, Ende: 7.72
was - Start: 7.72, Ende: 8.52
wollen - Start: 8.52, Ende: 8.72
Sie - Start: 8.72, Ende: 8.82
für - Start: 8.82, Ende: 8.98
die - Start: 8.98, Ende: 9.08
tun? - Start: 9.08, Ende: 9.34
Ich - Start: 10.14, Ende: 10.4
glaub, - Start: 10.4, Ende: 10.56
das - Start: 10.6, Ende: 10.72
Entscheidende - Start: 10.72, Ende: 11.34
ist - Start: 11.34, Ende: 11.6
b

In [ ]:
result["text"]

" Die hart arbeitende Mitte, über die wird ganz viel gesprochen in Deutschland. Wer ist diese hart arbeitende Mitte eigentlich? Und was wollen Sie für die tun? Ich glaub, das Entscheidende ist bei den Menschen, die wirklich auf die Arbeit gehen, die jeden Tag auch wollen, dass ihre Kinder eine gute Ausbildung bekommen. Da spielt Bildung die alles entscheidende Rolle. Dass sie je nach ihren Fähigkeiten in die unterschiedlichen Schularten gehen können, dass sie einen guten Abschluss bekommen. Dass sie die Möglichkeit haben, so wie das früher der Fall war, auch wenn das eine romantische Vorstellung ist, aber dass man die Möglichkeit hat, sich Eigentum zu schaffen. Mich beschäftigt sehr, dass wir in Europa die niedrigste Eigentumsrate haben. Das bedingt natürlich auch ganz viel zum Thema sozialer Frieden. Wenn ich früher einen Facharbeiter hatte, die Frau zu Hause war und Kinder da waren, konnte man ein Haus bauen. Heute können zwei Akademiker sich nicht mal eine Wohnung kaufen. Bildung sp

In [ ]:
# Extract the transcribed text
transcribed_text = result["text"]

with open("transcription_" + name + ".txt", "w", encoding="utf-8") as file:
    file.write(transcribed_text)

In [ ]:
# Merge speaker diarization and transcription
diarization_segments = []
min_duration = 1.0  #Minimum duration for a speach in seconds
tolerance = 0.2  # Allows small deviations for the first words

for segment, _, speaker in diarization.itertracks(yield_label=True):
    segment_duration = segment.end - segment.start
    if segment_duration >= min_duration:
        diarization_segments.append((segment.start, segment.end, speaker))

active_speaker = None
combined_output = []
merged_start = None
merged_end = None
merged_speaker = None
merged_text = []

# Check which speaker is assigned to each word in the transcription
for segment in result["segments"]:
    for word in segment["words"]:
        current_time = word["start"]

        # Find the matching speaker segment based on the timestamps
        closest_speaker = None
        for start, end, speaker in diarization_segments:
            # Check whether the word is within a segment or just before it (tolerance)
            if start - tolerance <= current_time <= end:
                closest_speaker = speaker
                break 

        # When a speaker has been found
        if closest_speaker is not None:
            # If the speaker changes or no speaker is set yet
            if active_speaker != closest_speaker:
                # If there has been a change of speaker, write the previous text
                if merged_speaker:
                    speaker_name = speaker_names.get(merged_speaker, "Unknown Speaker")
                    combined_output.append(f"{speaker_name}: {' '.join(merged_text)}")
                
                # Set variables for the new speaker
                merged_speaker = closest_speaker
                merged_start = word["start"]
                merged_end = word["end"]
                merged_text = [word["text"]]
                active_speaker = closest_speaker  # Update active speaker
            else:
                # If the speaker remains the same, add the word
                merged_end = word["end"]
                merged_text.append(word["text"])
        else:
            # If no speaker can be assigned, append the word to the last speaker
            if merged_speaker:
                merged_text.append(word["text"])

# Add last merged speaker, if present
if merged_speaker:
    speaker_name = speaker_names.get(merged_speaker, "Unknown Speaker")
    combined_output.append(f"{speaker_name}: {' '.join(merged_text)}")

# Save output
output_file = "combined_output" + name + ".txt"
with open(output_file, "w", encoding="utf-8") as f:
    for line in combined_output:
        f.write(line + "\n")

Combined output written to combined_output_2Hart_aber_fair_Vierkampf_timestamped.txt
